# Introdução à biblioteca Pandas

Pandas é uma biblioteca Python muito usada para manipulação e análise de dados. Nós iremos usar essa biblioteca nas atividades da disciplina. Vamos explicar neste notebook algumas das suas principais funções.

Para instalar a biblioteca Pandas voc o comando `pip install pandas`. Em seguida, carregue a biblioteca com o comando abaixo:

In [63]:
import pandas as pd

## Motivação

Suponha que você tem uma lista de notas de uma unidade de uma disciplina:

In [64]:
notas_lst = [10, 8.5, 5, 5]
notas_lst

[10, 8.5, 5, 5]

Uma forma tradicional de calcular a média das notas é através de um `for-loop`:

In [65]:
soma = 0
for nota in notas_lst:
    soma += nota

print(f"A média é {soma/len(notas_lst)}")

A média é 7.125


Nós vamos ver que à medida que precisamos manipular dados e calcular estatísticas em agrupamentos, o uso de `loops` se torna muito complexo. Veremos que a biblioteca Pandas apresenta muitas facilidades para manipulação e análise de dados através de suas estruturas de dados e funções aplicadas nos dados.



## Estruturas de dados Pandas

O Pandas tem duas estruturas de dados principais: _Series_ e _DataFrame_.

### Series

Uma _Series_ é um objeto unidimensional como um array, que contém uma sequência de valores do mesmo tipo.
Abaixo temos um exemplo da criação de _Series_ recebendo uma lista de notas como entrada:

In [66]:
notas = pd.Series([10, 8.5, 5, 5])
notas

0    10.0
1     8.5
2     5.0
3     5.0
dtype: float64

Um _Series_ também possui uma sequência de _labels_ chamada _index_, que identifica cada elemento da série. Por padrão o _index_ é um inteiro incremental de 0 à n-1, mas pode ser definido na sua criação. Por exemplo, adicionando o nome dos aluno relacionado a cada nota:

In [67]:
notas = pd.Series([10, 8.5, 5, 5], index=["john", "paul", "george", "ringo"])
notas

john      10.0
paul       8.5
george     5.0
ringo      5.0
dtype: float64

O Pandas pode aplicar funções em todos os dados de um série sem a necessidade de `loops`, como no exemplo abaixo que calcula a média:

In [68]:
notas.mean()

7.125

Também podemos aplicar filtros, extraindo por exemplo apenas as notas acima da média:

In [69]:
notas[notas >= 7]

john    10.0
paul     8.5
dtype: float64

Ou pegar a nota com um _index_ específico:

In [70]:
notas["paul"]

8.5

### DataFrame

Um _DataFrame_ representa dados como uma tabela retangular, com colunas e linhas que posuem nomes (_index_). Colunas diferentes podem ser de tipos diferentes (ex: string, int, float), mas para uma coluna específica todos os seus valores terão o mesmo tipo. Podemos pensar no _DataFrame_ como um dicionário de _Series_.

Uma das formas de criar um _DataFrame_ é passando um dicionário de listas de mesmo tamanho, como no exemplo abaixo:

In [71]:
notas_df = pd.DataFrame({
    "aluno": ["john", "paul", "george", "ringo"],
    "nota1": [10, 8.5, 5, 5],
    "nota2": [9, 10, 1, 4],
    "nota3": [8, 10, 3, 3.6]
})
notas_df

,aluno,nota1,nota2,nota3
0,john,10.0,9,8.0
1,paul,8.5,10,10.0
2,george,5.0,1,3.0
3,ringo,5.0,4,3.6


Para adicionar uma nova coluna com a média de cada aluno na disciplina, podemos aplicar a função [`mean`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html) nas colunas de notas do _DataFrame_. O parâmetro `axis=columns` indica que vamos calcular a média para colunas e não para linhas.

In [72]:
notas_df["media"] = notas_df[["nota1", "nota2", "nota3"]].mean(axis="columns")
notas_df

,aluno,nota1,nota2,nota3,media
0,john,10.0,9,8.0,9.0
1,paul,8.5,10,10.0,9.5
2,george,5.0,1,3.0,3.0
3,ringo,5.0,4,3.6,4.2


### Função `apply`

E se quisermos adicionar uma nova coluna que indica se um aluno foi aprovado (`nota >= 5`) ou reprovado? Podemos criar uma função que faça esse cálculo e usar a função [`apply`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) de um _DataFrame_ para aplicar para cada linha:

In [73]:
def calcula_resultado(media):
    return 'APROVADO' if media >= 5 else 'REPROVADO'
 

notas_df["resultado"] = notas_df["media"].apply(calcula_resultado)
notas_df

,aluno,nota1,nota2,nota3,media,resultado
0,john,10.0,9,8.0,9.0,APROVADO
1,paul,8.5,10,10.0,9.5,APROVADO
2,george,5.0,1,3.0,3.0,REPROVADO
3,ringo,5.0,4,3.6,4.2,REPROVADO


### Filtros

Podemos filtrar linhas tabela de acordo com alguma condição. Por exemplo, filtrando apenas os aprovados:

In [74]:
notas_df[notas_df["resultado"] == "APROVADO"]

,aluno,nota1,nota2,nota3,media,resultado
0,john,10.0,9,8.0,9.0,APROVADO
1,paul,8.5,10,10.0,9.5,APROVADO


Se quisermos calcular a média das notas dos alunos aprovados e dos reprovados? Uma primeira opção é aplicando filtros para cada grupo e em seguida calculando a média:

In [75]:
notas_df[notas_df["resultado"] == "APROVADO"].mean(numeric_only=True)

nota1    9.25
nota2    9.50
nota3    9.00
media    9.25
dtype: float64

In [76]:
notas_df[notas_df["resultado"] == "REPROVADO"].mean(numeric_only=True)

nota1    5.0
nota2    2.5
nota3    3.3
media    3.6
dtype: float64

### Group by

Nesse exemplo só temos dois grupos, então é relativamente fácil filtrar e aplicar a função em seguida. Quando temos muitos grupos, uma forma mais adequada é aplicando a função [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) para aplicar funções para os grupos definidos pelos valores de colunas da tabela. Exemplos:

In [77]:
notas_df.groupby(["resultado"]).mean()

,nota1,nota2,nota3,media
resultado,,,,
APROVADO,9.25,9.5,9.0,9.25
REPROVADO,5.00,2.5,3.3,3.60


### Ordenação

Podemos ordenar um _DataFrame_ com base em alguma coluna com a função [`sort_values`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html), por exemplo:

In [78]:
notas_df.sort_values(by="media")

,aluno,nota1,nota2,nota3,media,resultado
2,george,5.0,1,3.0,3.0,REPROVADO
3,ringo,5.0,4,3.6,4.2,REPROVADO
0,john,10.0,9,8.0,9.0,APROVADO
1,paul,8.5,10,10.0,9.5,APROVADO


Desta forma, se quisermos pegar o aluno com a maior média, podemos fazer ordenar o _DataFrame_ em ordem descendente (`ascending=False`) e obter o primeiro item com a função `.iloc[0]`.

In [79]:
notas_df.sort_values(by="media", ascending=False).iloc[0]

aluno            paul
nota1             8.5
nota2              10
nota3            10.0
media             9.5
resultado    APROVADO
Name: 1, dtype: object

# Referências

- [Python for Data Analysis - 5  Getting Started with pandas](https://wesmckinney.com/book/pandas-basics.html)